In [7]:
import socket
import threading
import json
import os

class Peer:
    def __init__(self, config_path):
        # Load configuration from file
        with open(config_path, 'r') as config_file:
            config = json.load(config_file)

        self.host = config['host']
        self.port = config['port']
        self.known_peers = config['known_peers']

        # Start server thread to listen for incoming connections
        server_thread = threading.Thread(target=self.start_server)
        server_thread.start()

    def start_server(self):
        server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        server_socket.bind((self.host, self.port))
        server_socket.listen(5)
        print(f"[INFO] Peer listening on {self.host}:{self.port}")

        while True:
            conn, addr = server_socket.accept()
            print(f"[INFO] Connection established with {addr}")

            # Start a thread to handle communication with each connected peer
            threading.Thread(target=self.handle_peer, args=(conn,)).start()

    def handle_peer(self, conn):
        # Receive file metadata (file name and size)
        file_metadata = conn.recv(1024).decode('utf-8')
        file_name, file_size = file_metadata.split(',')
        file_size = int(file_size)
        print(f"[INFO] Receiving file '{file_name}' of size {file_size} bytes")

        # Receive the file in chunks and save it
        with open(file_name, 'wb') as file:
            bytes_received = 0
            while bytes_received < file_size:
                chunk = conn.recv(1024)
                if not chunk:
                    break
                file.write(chunk)
                bytes_received += len(chunk)

        print(f"[INFO] File '{file_name}' received successfully")
        conn.close()

    def send_file(self, peer_host, peer_port, file_path):
        # Connect to the target peer
        client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        client_socket.connect((peer_host, peer_port))

        # Get file size
        file_size = os.path.getsize(file_path)
        file_name = os.path.basename(file_path)

        # Send file metadata (file name and size)
        client_socket.send(f"{file_name},{file_size}".encode('utf-8'))

        # Send the file in chunks
        with open(file_path, 'rb') as file:
            while chunk := file.read(1024):
                client_socket.send(chunk)

        print(f"[INFO] File '{file_name}' sent to {peer_host}:{peer_port}")
        client_socket.close()

# Example usage:
if __name__ == "__main__":
    # Initialize peers using their config files
    peer1 = Peer("peer1_config.json")
    peer2 = Peer("peer2_config.json")

    # Peer1 sends a file to Peer2
    file_to_send = "example.txt"  # This should be a file in the same directory
    peer1.send_file("127.0.0.1", 5002, file_to_send)


Exception in thread Thread-20 (start_server):
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-7-c727182e55f9>", line 22, in start_server
OSError: [Errno 98] Address already in use
Exception in thread Thread-21 (start_server):
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner


[INFO] Connection established with ('127.0.0.1', 49974)
[RECEIVED from ('127.0.0.1', 49974)]: example.txt,19
[INFO] File 'example.txt' sent to 127.0.0.1:5002


    self.run()
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-7-c727182e55f9>", line 22, in start_server
OSError: [Errno 98] Address already in use


[RECEIVED from ('127.0.0.1', 49974)]: hi this ia a mesage
[INFO] Connection closed with ('127.0.0.1', 49974)
